# Text Classification: Calibrating Sentiment Analysis Across Domains

This example demonstrates rank-preserving calibration for text classification, specifically sentiment analysis. We'll show how models trained on one domain (e.g., movie reviews) can be calibrated for deployment in different domains with different sentiment distributions.

## Business Motivation

Sentiment analysis models face several challenges when deployed across domains:
- **Domain shift**: Movie reviews vs. product reviews vs. social media posts
- **Sentiment distribution shift**: Different platforms have different baseline sentiment
- **Calibration issues**: Models are often overconfident in their predictions

Rank-preserving calibration helps maintain the relative ordering of texts by sentiment intensity while adjusting absolute probabilities to match the target domain's sentiment distribution.

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import spearmanr
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

# Import our calibration package - proper imports
from rank_preserving_calibration import calibrate_dykstra

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette(['#3498db', '#e74c3c', '#f39c12'])  # Blue, Red, Orange
np.random.seed(42)

print("📰 TEXT CLASSIFICATION CALIBRATION TOOLKIT LOADED")
print("Focus: Cross-domain sentiment analysis with rank preservation")

## Data Preparation: Simulating Multi-Domain Sentiment

We'll use the 20 Newsgroups dataset and create a sentiment classification task by treating different newsgroup categories as having different sentiment characteristics.

In [ ]:
# Load 20 newsgroups data for text classification
# We'll select categories that naturally have different "sentiment" characteristics
categories = [
    'alt.atheism',  # Often critical/negative discussions
    'rec.sport.baseball',  # Generally positive/enthusiastic
    'sci.space',  # Neutral/informative
    'talk.politics.misc',  # Mixed sentiment, often contentious
    'comp.graphics',  # Technical/neutral
    'rec.motorcycles'  # Enthusiastic/positive
]

# Load the data
newsgroups = fetch_20newsgroups(
    subset='all',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=('headers', 'footers', 'quotes')  # Remove metadata
)

print(f"Loaded {len(newsgroups.data)} documents from {len(categories)} categories")
print(f"Categories: {newsgroups.target_names}")
print(f"Category distribution: {Counter(newsgroups.target)}")

## Create Sentiment Labels and Domain Structure

We'll create a realistic sentiment classification scenario by:
1. Simulating sentiment labels based on category characteristics
2. Creating training and target domains with different sentiment distributions

In [ ]:
# Create sentiment mapping based on category characteristics
# This simulates how different domains have different baseline sentiment
category_sentiment_bias = {
    'alt.atheism': {'negative': 0.4, 'neutral': 0.4, 'positive': 0.2},
    'rec.sport.baseball': {'negative': 0.1, 'neutral': 0.3, 'positive': 0.6},
    'sci.space': {'negative': 0.1, 'neutral': 0.7, 'positive': 0.2},
    'talk.politics.misc': {'negative': 0.5, 'neutral': 0.3, 'positive': 0.2},
    'comp.graphics': {'negative': 0.2, 'neutral': 0.6, 'positive': 0.2},
    'rec.motorcycles': {'negative': 0.15, 'neutral': 0.25, 'positive': 0.6}
}

# Function to generate sentiment labels based on category
def generate_sentiment_labels(targets, target_names, sentiment_mapping, random_state=42):
    """Generate sentiment labels based on category bias."""
    np.random.seed(random_state)
    labels = []

    for target in targets:
        category = target_names[target]
        probs = sentiment_mapping[category]

        # Sample sentiment based on category bias
        sentiment = np.random.choice(
            ['negative', 'neutral', 'positive'],
            p=[probs['negative'], probs['neutral'], probs['positive']]
        )

        # Convert to numeric labels
        if sentiment == 'negative':
            labels.append(0)
        elif sentiment == 'neutral':
            labels.append(1)
        else:  # positive
            labels.append(2)

    return np.array(labels)

# Generate sentiment labels
sentiment_labels = generate_sentiment_labels(
    newsgroups.target, newsgroups.target_names, category_sentiment_bias
)

print("Sentiment Distribution:")
sentiment_counts = Counter(sentiment_labels)
sentiment_names = ['Negative', 'Neutral', 'Positive']
for i, name in enumerate(sentiment_names):
    count = sentiment_counts[i]
    pct = count / len(sentiment_labels) * 100
    print(f"  {name}: {count} ({pct:.1f}%)")

# Show distribution by category
print("\nSentiment by Category:")
for i, category in enumerate(newsgroups.target_names):
    mask = newsgroups.target == i
    cat_sentiments = sentiment_labels[mask]
    dist = [np.mean(cat_sentiments == j) for j in range(3)]
    print(f"  {category}: Neg={dist[0]:.2f}, Neu={dist[1]:.2f}, Pos={dist[2]:.2f}")

## Text Preprocessing and Feature Extraction

We'll create TF-IDF features for our text classification model.

In [ ]:
# Text preprocessing function
def preprocess_text(text):
    """Basic text preprocessing."""
    # Convert to lowercase
    text = text.lower()

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)

    # Remove very short texts (likely corrupted)
    if len(text.split()) < 5:
        return ""

    return text.strip()

# Preprocess all texts
processed_texts = [preprocess_text(text) for text in newsgroups.data]

# Filter out empty texts
valid_indices = [i for i, text in enumerate(processed_texts) if text != ""]
texts = [processed_texts[i] for i in valid_indices]
targets = newsgroups.target[valid_indices]
sentiments = sentiment_labels[valid_indices]

print(f"After preprocessing: {len(texts)} valid documents")
print(f"Average document length: {np.mean([len(text.split()) for text in texts]):.1f} words")

# Create TF-IDF features
vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95
)

X = vectorizer.fit_transform(texts)
print(f"Feature matrix shape: {X.shape}")
print(f"Feature matrix sparsity: {(1 - X.nnz / X.size) * 100:.1f}%")

## Model Training: Domain-Specific Sentiment Classifier

We'll train a sentiment classifier and then simulate deploying it to a domain with different sentiment distribution.

In [ ]:
# Split data into train and test
X_train, X_test, y_train, y_test, target_train, target_test = train_test_split(
    X, sentiments, targets, test_size=0.3, random_state=42, stratify=sentiments
)

print(f"Training set: {X_train.shape[0]} documents")
print(f"Test set: {X_test.shape[0]} documents")

# Training sentiment distribution
train_sentiment_dist = np.bincount(y_train) / len(y_train)
print("\nTraining sentiment distribution:")
for i, name in enumerate(sentiment_names):
    print(f"  {name}: {train_sentiment_dist[i]:.3f}")

# Train logistic regression classifier
clf = LogisticRegression(
    random_state=42,
    max_iter=1000,
    multi_class='ovr',  # One-vs-rest for interpretability
    C=1.0
)

clf.fit(X_train, y_train)

# Get predictions
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)

print("\nModel Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")

# Multi-class AUC (one-vs-rest)
auc_scores = []
for i in range(3):
    y_binary = (y_test == i).astype(int)
    auc = roc_auc_score(y_binary, y_proba[:, i])
    auc_scores.append(auc)
    print(f"AUC {sentiment_names[i]}: {auc:.3f}")

print(f"Mean AUC: {np.mean(auc_scores):.3f}")

# Show predicted vs actual sentiment distribution
test_sentiment_dist = np.bincount(y_test) / len(y_test)
pred_sentiment_dist = y_proba.mean(axis=0)

print("\nTest set sentiment distribution:")
print(f"  Actual: {test_sentiment_dist}")
print(f"  Predicted: {pred_sentiment_dist}")

## Simulation: Domain Shift Scenario

We'll simulate deploying our model to a new domain (e.g., social media) with a different sentiment distribution.

In [ ]:
# Simulate target domain with different sentiment distribution
# Example: Social media platform with more positive content
target_domain = "Social Media Platform"
target_sentiment_distribution = np.array([0.20, 0.25, 0.55])  # More positive sentiment

print(f"Domain Shift Scenario: Deploying to {target_domain}")
print("\nSentiment Distribution Comparison:")
print(f"{'Sentiment':<12} {'Training':<12} {'Target':<12} {'Difference':<12}")
print("-" * 50)

for i, name in enumerate(sentiment_names):
    diff = target_sentiment_distribution[i] - train_sentiment_dist[i]
    print(f"{name:<12} {train_sentiment_dist[i]:<12.3f} {target_sentiment_distribution[i]:<12.3f} {diff:<+12.3f}")

# Calculate target marginals for calibration
n_samples = len(X_test)
target_marginals = target_sentiment_distribution * n_samples

print(f"\nTarget marginals for {n_samples} samples:")
for i, name in enumerate(sentiment_names):
    print(f"  {name}: {target_marginals[i]:.1f} samples")

print("\nCurrent model predictions:")
current_marginals = y_proba.sum(axis=0)
for i, name in enumerate(sentiment_names):
    print(f"  {name}: {current_marginals[i]:.1f} samples")

print("\nMarginal adjustments needed:")
marginal_diffs = target_marginals - current_marginals
for i, name in enumerate(sentiment_names):
    print(f"  {name}: {marginal_diffs[i]:+.1f} samples")

## Apply Rank-Preserving Calibration

Now we'll calibrate the model's predictions to match the target domain's sentiment distribution while preserving the relative ranking of texts by sentiment intensity.

In [ ]:
# Apply rank-preserving calibration
print("Applying rank-preserving calibration...")
result = calibrate_dykstra(y_proba, target_marginals, verbose=True, max_iters=3000)

print("\nCalibration Results:")
print(f"Converged: {result.converged}")
print(f"Iterations: {result.iterations}")
print(f"Final change: {result.final_change:.2e}")
print(f"Max row error: {result.max_row_error:.2e}")
print(f"Max column error: {result.max_col_error:.2e}")
print(f"Max rank violation: {result.max_rank_violation:.2e}")

# Get calibrated probabilities
y_proba_calibrated = result.Q

# Verify calibration worked
calibrated_marginals = y_proba_calibrated.sum(axis=0)
print("\nVerification - Calibrated marginals:")
for i, name in enumerate(sentiment_names):
    error = abs(calibrated_marginals[i] - target_marginals[i])
    print(f"  {name}: {calibrated_marginals[i]:.1f} (target: {target_marginals[i]:.1f}, error: {error:.1e})")

# Check rank preservation
rank_preservation_check = True
for j in range(3):
    original_order = np.argsort(y_proba[:, j])
    calibrated_order = np.argsort(y_proba_calibrated[:, j])
    if not np.array_equal(original_order, calibrated_order):
        rank_preservation_check = False
        break

print(f"\nRank preservation: {'✓ Perfect' if rank_preservation_check else '⚠ Approximate'}")

# Calculate sentiment distribution changes
original_mean_probs = y_proba.mean(axis=0)
calibrated_mean_probs = y_proba_calibrated.mean(axis=0)

print("\nMean probability changes:")
for i, name in enumerate(sentiment_names):
    change = calibrated_mean_probs[i] - original_mean_probs[i]
    print(f"  {name}: {original_mean_probs[i]:.3f} → {calibrated_mean_probs[i]:.3f} ({change:+.3f})")

## Comprehensive Visualization: Before and After Calibration

Let's visualize the impact of calibration on sentiment predictions.

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Text Classification: Impact of Rank-Preserving Calibration', fontsize=16, y=0.98)

colors = ['#e74c3c', '#f39c12', '#2ecc71']  # Red, Orange, Green for Negative, Neutral, Positive

# 1. Sentiment distribution comparison
x_pos = np.arange(3)
width = 0.25

axes[0, 0].bar(x_pos - width, train_sentiment_dist, width, label='Training', alpha=0.8, color=colors)
axes[0, 0].bar(x_pos, original_mean_probs, width, label='Original Model', alpha=0.8)
axes[0, 0].bar(x_pos + width, target_sentiment_distribution, width, label='Target Domain', alpha=0.8)
axes[0, 0].set_xlabel('Sentiment Class')
axes[0, 0].set_ylabel('Proportion')
axes[0, 0].set_title('Sentiment Distribution Comparison')
axes[0, 0].set_xticks(x_pos)
axes[0, 0].set_xticklabels(sentiment_names)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Probability transformation for each class
for i, (name, color) in enumerate(zip(sentiment_names, colors, strict=False)):
    axes[0, 1].scatter(y_proba[:, i], y_proba_calibrated[:, i], alpha=0.6, s=20, color=color, label=name)

axes[0, 1].plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Identity')
axes[0, 1].set_xlabel('Original Probability')
axes[0, 1].set_ylabel('Calibrated Probability')
axes[0, 1].set_title('Probability Transformation by Class')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Distribution of probability changes
prob_changes = y_proba_calibrated - y_proba
for i, (name, color) in enumerate(zip(sentiment_names, colors, strict=False)):
    axes[0, 2].hist(prob_changes[:, i], bins=30, alpha=0.7, label=name, color=color, density=True)

axes[0, 2].axvline(0, color='black', linestyle='--', alpha=0.5)
axes[0, 2].set_xlabel('Probability Change (Calibrated - Original)')
axes[0, 2].set_ylabel('Density')
axes[0, 2].set_title('Distribution of Probability Changes')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# 4. Confidence distribution changes
max_probs_original = np.max(y_proba, axis=1)
max_probs_calibrated = np.max(y_proba_calibrated, axis=1)

axes[1, 0].hist(max_probs_original, bins=30, alpha=0.7, label='Original', density=True)
axes[1, 0].hist(max_probs_calibrated, bins=30, alpha=0.7, label='Calibrated', density=True)
axes[1, 0].set_xlabel('Maximum Probability (Confidence)')
axes[1, 0].set_ylabel('Density')
axes[1, 0].set_title('Confidence Distribution')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 5. Rank preservation visualization
# Show correlation between original and calibrated probabilities

correlations = []
for i, (name, color) in enumerate(zip(sentiment_names, colors, strict=False)):
    corr, _ = spearmanr(y_proba[:, i], y_proba_calibrated[:, i])
    correlations.append(corr)

    # Show top and bottom examples for this class
    top_indices = np.argsort(y_proba[:, i])[-10:]
    axes[1, 1].scatter(y_proba[top_indices, i], y_proba_calibrated[top_indices, i],
                      color=color, alpha=0.8, s=50, label=f'{name} (top 10)')

axes[1, 1].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[1, 1].set_xlabel('Original Probability')
axes[1, 1].set_ylabel('Calibrated Probability')
axes[1, 1].set_title('Rank Preservation (Top Predictions)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# 6. Decision boundary changes
thresholds = np.linspace(0.1, 0.9, 9)
decision_changes_by_threshold = []

for thresh in thresholds:
    original_decisions = np.argmax(y_proba, axis=1)
    calibrated_decisions = np.argmax(y_proba_calibrated, axis=1)

    # Count confident predictions above threshold
    original_confident = np.max(y_proba, axis=1) > thresh
    calibrated_confident = np.max(y_proba_calibrated, axis=1) > thresh

    # Count decision changes among confident predictions
    confident_mask = original_confident | calibrated_confident
    if np.any(confident_mask):
        changes = np.sum((original_decisions != calibrated_decisions) & confident_mask)
        total_confident = np.sum(confident_mask)
        change_rate = changes / total_confident * 100
    else:
        change_rate = 0

    decision_changes_by_threshold.append(change_rate)

axes[1, 2].plot(thresholds, decision_changes_by_threshold, 'o-', linewidth=2, markersize=8)
axes[1, 2].set_xlabel('Confidence Threshold')
axes[1, 2].set_ylabel('Decision Changes (%)')
axes[1, 2].set_title('Impact on High-Confidence Decisions')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print correlation summary
print("\nRank Preservation Summary (Spearman Correlations):")
for i, (name, corr) in enumerate(zip(sentiment_names, correlations, strict=False)):
    print(f"  {name}: {corr:.6f}")
print(f"  Mean: {np.mean(correlations):.6f}")

## Performance Analysis: Model Quality Metrics

Let's evaluate how calibration affects various performance metrics.

In [ ]:
# Performance metrics comparison
from sklearn.metrics import accuracy_score, f1_score, log_loss


def calculate_multiclass_metrics(y_true, y_proba):
    """Calculate various multiclass performance metrics."""
    y_pred = np.argmax(y_proba, axis=1)

    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'log_loss': log_loss(y_true, y_proba),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted')
    }

    # Per-class AUC
    auc_scores = []
    for i in range(y_proba.shape[1]):
        y_binary = (y_true == i).astype(int)
        auc = roc_auc_score(y_binary, y_proba[:, i])
        auc_scores.append(auc)

    metrics['auc_macro'] = np.mean(auc_scores)
    metrics['auc_per_class'] = auc_scores

    return metrics

def expected_calibration_error(y_true, y_proba, n_bins=10):
    """Calculate Expected Calibration Error for multiclass."""
    y_pred = np.argmax(y_proba, axis=1)
    confidences = np.max(y_proba, axis=1)
    accuracies = (y_pred == y_true).astype(float)

    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    ece = 0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers, strict=False):
        in_bin = (confidences > bin_lower) & (confidences <= bin_upper)
        prop_in_bin = in_bin.mean()

        if prop_in_bin > 0:
            accuracy_in_bin = accuracies[in_bin].mean()
            avg_confidence_in_bin = confidences[in_bin].mean()
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

    return ece

# Calculate metrics for original and calibrated predictions
metrics_original = calculate_multiclass_metrics(y_test, y_proba)
metrics_calibrated = calculate_multiclass_metrics(y_test, y_proba_calibrated)

ece_original = expected_calibration_error(y_test, y_proba)
ece_calibrated = expected_calibration_error(y_test, y_proba_calibrated)

# Create comparison table
print("Performance Metrics Comparison:")
print("=" * 70)
print(f"{'Metric':<20} {'Original':<12} {'Calibrated':<12} {'Change':<12}")
print("-" * 70)

for metric in ['accuracy', 'log_loss', 'f1_macro', 'f1_weighted', 'auc_macro']:
    orig_val = metrics_original[metric]
    cal_val = metrics_calibrated[metric]
    change = cal_val - orig_val
    print(f"{metric:<20} {orig_val:<12.4f} {cal_val:<12.4f} {change:<+12.4f}")

print(f"{'ECE':<20} {ece_original:<12.4f} {ece_calibrated:<12.4f} {ece_calibrated-ece_original:<+12.4f}")

print("\nPer-Class AUC Comparison:")
print(f"{'Class':<12} {'Original':<12} {'Calibrated':<12} {'Change':<12}")
print("-" * 50)
for i, name in enumerate(sentiment_names):
    orig_auc = metrics_original['auc_per_class'][i]
    cal_auc = metrics_calibrated['auc_per_class'][i]
    change = cal_auc - orig_auc
    print(f"{name:<12} {orig_auc:<12.4f} {cal_auc:<12.4f} {change:<+12.4f}")

print("\n📊 Key Observations:")
print(f"• Discrimination preserved: AUC change = {metrics_calibrated['auc_macro'] - metrics_original['auc_macro']:+.6f}")
print(f"• Calibration improved: ECE reduced by {ece_original - ece_calibrated:.4f}")
print(f"• Accuracy {'improved' if metrics_calibrated['accuracy'] > metrics_original['accuracy'] else 'maintained'}: {metrics_original['accuracy']:.4f} → {metrics_calibrated['accuracy']:.4f}")
print(f"• Domain adaptation: Distribution shifted to match target ({target_domain})")

## Real-World Application: Content Moderation Analysis

Let's demonstrate how this calibration would work in a practical content moderation scenario.

In [ ]:
# Content moderation scenario analysis
def analyze_content_moderation_impact(y_proba_orig, y_proba_cal, sentiment_threshold=0.7):
    """Analyze impact on content moderation decisions."""

    # Original moderation decisions
    # Flag content with high negative sentiment probability
    flag_negative_orig = y_proba_orig[:, 0] > sentiment_threshold  # Negative sentiment > threshold
    flag_negative_cal = y_proba_cal[:, 0] > sentiment_threshold

    # Promote content with high positive sentiment probability
    promote_positive_orig = y_proba_orig[:, 2] > sentiment_threshold  # Positive sentiment > threshold
    promote_positive_cal = y_proba_cal[:, 2] > sentiment_threshold

    results = {
        'flagged_negative': {
            'original': np.sum(flag_negative_orig),
            'calibrated': np.sum(flag_negative_cal),
            'changes': np.sum(flag_negative_orig != flag_negative_cal)
        },
        'promoted_positive': {
            'original': np.sum(promote_positive_orig),
            'calibrated': np.sum(promote_positive_cal),
            'changes': np.sum(promote_positive_orig != promote_positive_cal)
        }
    }

    return results

# Analyze at different thresholds
thresholds_to_test = [0.5, 0.6, 0.7, 0.8, 0.9]
moderation_results = []

for thresh in thresholds_to_test:
    result = analyze_content_moderation_impact(y_proba, y_proba_calibrated, thresh)
    result['threshold'] = thresh
    moderation_results.append(result)

# Display results
print(f"Content Moderation Impact Analysis (Target Domain: {target_domain})")
print("=" * 80)
print(f"Total content pieces: {len(y_test)}")
print(f"Target sentiment distribution: Negative={target_sentiment_distribution[0]:.1%}, Neutral={target_sentiment_distribution[1]:.1%}, Positive={target_sentiment_distribution[2]:.1%}")
print()

print("Negative Content Flagging:")
print(f"{'Threshold':<10} {'Original':<10} {'Calibrated':<12} {'Changes':<10} {'Change %':<10}")
print("-" * 60)
for result in moderation_results:
    thresh = result['threshold']
    orig = result['flagged_negative']['original']
    cal = result['flagged_negative']['calibrated']
    changes = result['flagged_negative']['changes']
    change_pct = changes / len(y_test) * 100
    print(f"{thresh:<10.1f} {orig:<10} {cal:<12} {changes:<10} {change_pct:<10.1f}%")

print("\nPositive Content Promotion:")
print(f"{'Threshold':<10} {'Original':<10} {'Calibrated':<12} {'Changes':<10} {'Change %':<10}")
print("-" * 60)
for result in moderation_results:
    thresh = result['threshold']
    orig = result['promoted_positive']['original']
    cal = result['promoted_positive']['calibrated']
    changes = result['promoted_positive']['changes']
    change_pct = changes / len(y_test) * 100
    print(f"{thresh:<10.1f} {orig:<10} {cal:<12} {changes:<10} {change_pct:<10.1f}%")

# Show sample texts affected by calibration
thresh_example = 0.7
flag_changes = (y_proba[:, 0] > thresh_example) != (y_proba_calibrated[:, 0] > thresh_example)
promo_changes = (y_proba[:, 2] > thresh_example) != (y_proba_calibrated[:, 2] > thresh_example)

print(f"\n📝 Example Content Affected by Calibration (threshold={thresh_example}):")

if np.any(flag_changes):
    changed_indices = np.where(flag_changes)[0][:3]  # Show first 3 examples
    print("\nNegative Content Flagging Changes:")
    for idx in changed_indices:
        text = texts[list(X_test.indices)[idx] if hasattr(X_test, 'indices') else idx][:100]
        orig_prob = y_proba[idx, 0]
        cal_prob = y_proba_calibrated[idx, 0]
        action = "Flagged" if cal_prob > thresh_example else "Unflagged"
        print(f"   Text: \"{text}...\"")
        print(f"   Negative prob: {orig_prob:.3f} → {cal_prob:.3f} ({action})")
        print()

if np.any(promo_changes):
    changed_indices = np.where(promo_changes)[0][:3]  # Show first 3 examples
    print("Positive Content Promotion Changes:")
    for idx in changed_indices:
        text = texts[list(X_test.indices)[idx] if hasattr(X_test, 'indices') else idx][:100]
        orig_prob = y_proba[idx, 2]
        cal_prob = y_proba_calibrated[idx, 2]
        action = "Promoted" if cal_prob > thresh_example else "Not promoted"
        print(f"   Text: \"{text}...\"")
        print(f"   Positive prob: {orig_prob:.3f} → {cal_prob:.3f} ({action})")
        print()

## Business Impact Summary

Let's summarize the business value of rank-preserving calibration for text classification.

In [ ]:
print("BUSINESS IMPACT SUMMARY: Text Classification Calibration")
print("=" * 80)

print("\n🎯 SCENARIO:")
print("   Model trained on newsgroup data (varied sentiment distribution)")
print(f"   Deployed to {target_domain} with {target_sentiment_distribution[2]:.0%} positive content")
print("   Need to maintain content ranking while adjusting sentiment probabilities")

print("\n📊 CALIBRATION RESULTS:")
original_dist = y_proba.mean(axis=0)
calibrated_dist = y_proba_calibrated.mean(axis=0)
print("   ✓ Sentiment distribution adjusted:")
for i, name in enumerate(sentiment_names):
    print(f"     {name}: {original_dist[i]:.3f} → {calibrated_dist[i]:.3f} (target: {target_sentiment_distribution[i]:.3f})")
print(f"   ✓ Ranking preserved: Mean Spearman correlation = {np.mean(correlations):.6f}")
print(f"   ✓ Discrimination maintained: AUC change = {metrics_calibrated['auc_macro'] - metrics_original['auc_macro']:+.6f}")

print("\n💼 BUSINESS BENEFITS:")
print("   • Accurate content prioritization for target audience")
print("   • Consistent user experience across different content sources")
print("   • Improved content moderation efficiency")
print("   • Better alignment with platform-specific engagement patterns")

# Calculate economic impact example
baseline_engagement = 0.15  # 15% baseline engagement rate
positive_boost = 1.3  # 30% boost from positive content
negative_penalty = 0.7  # 30% penalty from negative content

# Estimate engagement impact
orig_positive_weight = original_dist[2]
orig_negative_weight = original_dist[0]
cal_positive_weight = calibrated_dist[2]
cal_negative_weight = calibrated_dist[0]

orig_engagement_factor = (orig_positive_weight * positive_boost +
                         orig_negative_weight * negative_penalty +
                         original_dist[1] * 1.0)
cal_engagement_factor = (cal_positive_weight * positive_boost +
                        cal_negative_weight * negative_penalty +
                        calibrated_dist[1] * 1.0)

engagement_improvement = (cal_engagement_factor / orig_engagement_factor - 1) * 100

print("\n📈 ESTIMATED ENGAGEMENT IMPACT:")
print("   • Content sentiment better matches target audience preferences")
print(f"   • Estimated engagement improvement: {engagement_improvement:+.1f}%")
print("   • Reduced over-moderation of content in positive-leaning platform")
print("   • Better content recommendation relevance")

print("\n🔧 IMPLEMENTATION CONSIDERATIONS:")
print("   • Requires reliable estimates of target domain sentiment distribution")
print("   • Should be monitored for drift over time")
print("   • Consider A/B testing to validate engagement improvements")
print("   • May need periodic recalibration as content patterns evolve")

print("\n🎯 USE CASES FOR TEXT CLASSIFICATION CALIBRATION:")
print("   • Cross-platform content deployment (Twitter → LinkedIn → Reddit)")
print("   • International market adaptation (cultural sentiment differences)")
print("   • Content personalization at scale")
print("   • Multi-brand content strategy alignment")
print("   • Seasonal campaign optimization")

# Show final calibration quality
marginal_errors = np.abs(y_proba_calibrated.sum(axis=0) - target_marginals)
max_marginal_error = np.max(marginal_errors)
print("\n✅ CALIBRATION QUALITY:")
print(f"   • Maximum marginal error: {max_marginal_error:.2e}")
print(f"   • Converged in {result.iterations} iterations")
print("   • All constraints satisfied within numerical precision")

## Next Steps

This example demonstrated rank-preserving calibration for text classification across domains. The approach is broadly applicable to:

- **Multi-language sentiment analysis** with language-specific sentiment distributions
- **Topic classification** across different content sources
- **Spam detection** with varying spam rates across platforms
- **Content recommendation** systems with audience-specific preferences
- **Brand monitoring** across different social media platforms

For more real-world examples, see the other notebooks in this series:
- Medical diagnosis with population-specific disease prevalence
- Image classification with domain adaptation
- Financial risk assessment with portfolio-specific distributions
- Survey reweighting for demographic representation